## importing the model

In [1]:
from cobra.io import read_sbml_model, write_sbml_model
model = read_sbml_model('/Users/wagner/github-classroom/27410/group-assingment-team-4/yeast-GEM-8.7.0/model/yeast-GEM2.xml') # loading
from cobra import Reaction, Metabolite, Gene



# Finding the most suitable carbon source and testing different types of media

First we can test the production of beta-farnesene in aerobic conditions as well as anaerobic conditions:

In [2]:
model.reactions.query('oxygen exchange', 'name')

[<Reaction r_1992 at 0x28f0c6490>]

In [3]:
with model:
   medium = model.medium
   medium['r_1992'] = 0
   model.medium = medium
   with model:
      model.objective = model.reactions.BFD
      BF_production = model.optimize().objective_value
print(BF_production)

0.04739229024943303


There is no production in this case no point in looking further at that, so we will go back to aerobic conditions:

In [9]:
with model:
    growth_minimal_medium = model.optimize().objective_value
    print('Growth rate on minimal medium:', growth_minimal_medium, 'gDW h^-1')
    model.objective = model.reactions.BFD
    BF_production_minimal_medium = model.optimize().objective_value
    print("Maximum beta-farnesene production on minimal medium [mmol-beta-farnesene gDW^-1 h^-1]:", BF_production_minimal_medium)
    print("Theoretical maximum yield on minimal medium [mmol-beta-farnesene / mmol-glucose]:", BF_production_minimal_medium / (-1*model.reactions.r_1714.flux))
    max_theoretical_yield_minimal_medium = BF_production / (-1*model.reactions.r_1714.flux)

Growth rate on minimal medium: 0.08192815852363812 gDW h^-1
Maximum beta-farnesene production on minimal medium [mmol-beta-farnesene gDW^-1 h^-1]: 0.23035112219719728
Theoretical maximum yield on minimal medium [mmol-beta-farnesene / mmol-glucose]: 0.23035112219719728


Now we will test to see the production of beta-farnesene with increased glucose concentrations:

In [5]:
# First we will here make an overview of the full medium composition as it is in the model by default with
# growth on glucose, to do this we check through all the extracellular exchange reactions:

for component in model.medium.items():
    print(model.reactions.get_by_id(component[0]), "|", model.reactions.get_by_id(component[0]).name, "| concentration =", component[1])

r_1654: s_0420 <=>  | ammonium exchange | concentration = 1000.0
r_1714: s_0565 <=>  | D-glucose exchange | concentration = 1.0
r_1832: s_0796 <=>  | H+ exchange | concentration = 1000.0
r_1861: s_0925 <=>  | iron(2+) exchange | concentration = 1000.0
r_1992: s_1277 <=>  | oxygen exchange | concentration = 1000.0
r_2005: s_1324 <=>  | phosphate exchange | concentration = 1000.0
r_2020: s_1374 <=>  | potassium exchange | concentration = 1000.0
r_2049: s_1438 <=>  | sodium exchange | concentration = 1000.0
r_2060: s_1468 <=>  | sulphate exchange | concentration = 1000.0
r_2100: s_0805 <=>  | water exchange | concentration = 1000.0
r_4593: s_4200 <=>  | chloride exchange | concentration = 1000.0
r_4594: s_4201 <=>  | Cu2(+) exchange | concentration = 1000.0
r_4595: s_4202 <=>  | Mn(2+) exchange | concentration = 1000.0
r_4596: s_4203 <=>  | Zn(2+) exchange | concentration = 1000.0
r_4597: s_4204 <=>  | Mg(2+) exchange | concentration = 1000.0
r_4600: s_4199 <=>  | Ca(2+) exchange | concen

This is a full overview of the medium composition. Using this we will now check the impact of changing the concentration of glucose in the medium on production and on growth:

In [59]:
# Testing different glucose concentrations of 0.1X, 10X, 100X and 1000X the standard glucose concentration in the medium

medium = model.medium
with model:
   medium['r_1714'] = 0.1
   model.medium = medium
   point_one_X_glc_growth = model.optimize().objective_value
   model.objective = model.reactions.BFD
   point_one_X_glc_productivity = model.optimize().objective_value
   print('Maximal growth with 0.1X glucose:', point_one_X_glc_growth)
   print('Maximal production of beta-farnesene with 0.1X glucose:', point_one_X_glc_productivity)
   print('Theoretical maximum beta-farnesene production on 0.1X glucose:', model.optimize().objective_value / (-1*model.reactions.r_1714.flux))

print('')

with model:
   medium['r_1714'] = 10
   model.medium = medium
   ten_X_glc_growth = model.optimize().objective_value
   model.objective = model.reactions.BFD
   ten_X_glc_productivity = model.optimize().objective_value
   print('Maximal growth with 10X glucose:', ten_X_glc_growth)
   print('Maximal production of beta-farnesene with 10X glucose:', ten_X_glc_productivity)
   print('Theoretical maximum beta-farnesene production on 10X glucose:', model.optimize().objective_value / (-1*model.reactions.r_1714.flux))

print('')

with model:
   medium['r_1714'] = 100
   model.medium = medium
   hundred_X_glc_growth = model.optimize().objective_value
   model.objective = model.reactions.BFD
   hundred_X_glc_productivity = model.optimize().objective_value
   print('Maximal growth with 100X glucose:', hundred_X_glc_growth)
   print('Maximal production of beta-farnesene with 100X glucose:', hundred_X_glc_productivity)
   print('Theoretical maximum beta-farnesene production on 100X glucose:', model.optimize().objective_value / (-1*model.reactions.r_1714.flux))

print('')

with model:
   medium['r_1714'] = 1000
   model.medium = medium
   thousand_X_glc_growth = model.optimize().objective_value
   model.objective = model.reactions.BFD
   thousand_X_glc_productivity = model.optimize().objective_value
   print('Maximal growth with 1000X glucose:', thousand_X_glc_growth)
   print('Maximal production of beta-farnesene with 1000X glucose:', thousand_X_glc_productivity)
   print('Theoretical maximum beta-farnesene production on 1000X glucose:', model.optimize().objective_value / (-1*model.reactions.r_1714.flux))



   

Maximal growth with 0.1X glucose: 0.005401624710187918
Maximal production of beta-farnesene with 0.1X glucose: 0.01666211119007269
Theoretical maximum beta-farnesene production on 0.1X glucose: 0.1666211119007262

Maximal growth with 10X glucose: 0.8471930218949543
Maximal production of beta-farnesene with 10X glucose: 2.3416532799479137
Theoretical maximum beta-farnesene production on 10X glucose: 0.2341653279947901

Maximal growth with 100X glucose: 8.499842132127737
Maximal production of beta-farnesene with 100X glucose: 23.45467485745497
Theoretical maximum beta-farnesene production on 100X glucose: 0.23454674857454985

Maximal growth with 1000X glucose: 19.81785961671279
Maximal production of beta-farnesene with 1000X glucose: 76.91806853582865
Theoretical maximum beta-farnesene production on 1000X glucose: 0.11999593706928555


It seems that there is little to no benefit in the theoretical maximum of beta-farnesene production neither increasing nor decreasing the glucose concentrations.


# Changing to a medium composition that mimicks YPD medium

For further medium optimization we will now look into changing the medium composition from the standard minimal medium of the model to the complete medium for yeast growth YPD medium also known as YPED medium.

In [32]:
YPD_components = ['Glucose', 'Alanine', 'Arginine', 'Aspartic', 'Cysteine', 'Glutamine', 'Glycine', 'Histidine', 'Isoleucine',
                  'Leucine', 'Lysine', 'Methionine', 'Phenylalanine', 'Proline', 'Serine', 'Threonine', 'Tryptophan',
                  'Tyrosine', 'Valine', 'Zinc', 'FE2']

YPD_medium = model.medium

YPD_medium['r_1714'] = 20
YPD_medium['r_1873'] = 10
YPD_medium['r_1879'] = 10
YPD_medium['r_1881'] = 10
YPD_medium['r_1883'] = 10
YPD_medium['r_1891'] = 10
YPD_medium['r_1810'] = 10
YPD_medium['r_1893'] = 10
YPD_medium['r_1897'] = 10
YPD_medium['r_1899'] = 10
YPD_medium['r_1900'] = 10
YPD_medium['r_1902'] = 10
YPD_medium['r_1903'] = 10
YPD_medium['r_1904'] = 10
YPD_medium['r_1906'] = 10
YPD_medium['r_1911'] = 10
YPD_medium['r_1912'] = 10
YPD_medium['r_1913'] = 10
YPD_medium['r_1914'] = 10
YPD_medium['r_4596'] = 10
YPD_medium['r_1861'] = 10

model.objective = model.reactions.r_2111
model.medium = YPD_medium
YPD_growth = model.optimize().objective_value
model.objective = model.reactions.BFD
YPD_productivity = model.optimize().objective_value
print('Maximal growth with YPD medium:', YPD_growth, 'gDW h^-1')
print('Maximal theoretical production of beta-farnesene with YPD medium:', YPD_productivity, 'mmol gDW^-1 h^-1')
print('Theoretical maximum yield of beta-farnesene on YPD medium:', 
      model.optimize().objective_value / (-1*model.reactions.r_1714.flux), 'mmol-BF mmol-glc^-1')
   

Maximal growth with YPD medium: 9.203912627951905 gDW h^-1
Maximal theoretical production of beta-farnesene with YPD medium: 20.445907330275233 mmol gDW^-1 h^-1
Theoretical maximum yield of beta-farnesene on YPD medium: 1.0222953665137653 mmol-BF mmol-glc^-1


# Saving the model with new medium composition (YPD medium)

In [22]:
# This code saves the changes to the model and we now have a model that has a the heterologous pathway for 
# beta-farnesene production as well as a demand reaction for cytoplasmic beta-farnesene

write_sbml_model(model,
                 filename = '/Users/wagner/github-classroom/27410/group-assingment-team-4/yeast-GEM-8.7.0/model/yeast-GEM3.xml'
                 )  
#saving the model